In [2]:
import torch
import torchvision
import models
from PIL import Image
import numpy as np
import pandas as pd
import sys
import os
from scipy import stats
import matplotlib.pyplot as plt
import cv2
from turtle import color
from sklearn.metrics import mean_squared_error 

def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

In [3]:
# model_hyper = models.HyperNet(16, 112, 224, 112, 56, 28, 14, 7).cuda()
model_hyper = models.HyperNet(16, 112, 224, 112, 56, 28, 14, 7).cpu()
model_hyper.train(False)
# load our pre-trained model on the koniq-10k dataset
ckpt = (torch.load('./pretrained/koniq_pretrained.pkl', map_location=torch.device('cpu')))
model_hyper.load_state_dict(ckpt)

transforms = torchvision.transforms.Compose([
                    torchvision.transforms.Resize((512, 384)),
                    torchvision.transforms.RandomCrop(size=224),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                                     std=(0.229, 0.224, 0.225))])

In [4]:
# 이미지가 있는 폴더 경로 지정
# filenames에 폴더 안의 이미지 파일명이 리스트로 저장된다.
# data 폴더 안에 예시 이미지 3장이 있음.
# data 폴더 안에 있는 예시 이미지 3장을 지우고 예측하려는 이미지를 복사 붙여넣기 하거나
# 아예 경로를 바꿀 것
filenames = os.listdir('./data')

In [5]:
# 이미지 파일명 확인
print(filenames)

['D_01.jpg', 'D_02.jpg', 'D_03.jpg']


In [31]:
# DataFrame을 만들어 준다. 
df = pd.DataFrame({'Model_name':['HyperIQA' for i in range(len(filenames))],
                        'filename':[filename for filename in filenames],
                        'score':[0 for i in range(len(filenames))]})
df

,Model_name,filename,score
0,HyperIQA,D_01.jpg,0
1,HyperIQA,D_02.jpg,0
2,HyperIQA,D_03.jpg,0


In [32]:
# for문을 이용해서 이미지 파일명을 하나하나 불러온다.
cnt = 0 # 몇 번째 사진인지 출력하기 위한 count
for filename in filenames:
    
    im_path = f'./data/{filename}' # {filename} 이거 앞에 폴더 경로를 위에서 넣어준 것과 같게 변경해야 한다.
    # random crop 10 patches and calculate mean quality score
    pred_scores = []
    for i in range(10):
        img = pil_loader(im_path)
        img = transforms(img)
        # img = torch.tensor(img.cuda()).unsqueeze(0)
        img = torch.tensor(img.cpu()).unsqueeze(0)
        paras = model_hyper(img)  # 'paras' contains the network weights conveyed to target network

        # Building target network
        # model_target = models.TargetNet(paras).cuda()
        model_target = models.TargetNet(paras).cpu()
        for param in model_target.parameters():
            param.requires_grad = False

        # Quality prediction
        pred = model_target(paras['target_in_vec'])  # 'paras['target_in_vec']' is the input to target net
        pred_scores.append(float(pred.item()))
    score = np.mean(pred_scores) # 이게 최종 예측 score 임.
    
    # 아래 코드는 DataFrame을 만들 경우 score 값을 바로 DataFrame에 넣기 위한 코드이다.
    df.loc[df['filename'] == filename, 'score'] = score
    # df['filename'] = filename
    # df['score'] = score
    # df.loc[df.index == cnt, 'score'] = score
    # 만약 원하는 이미지 장수만큼만 예측할 경우 count를 이용해서 break 해준다.
    # 폴더 내 모든 이미지를 예측할 경우 주석 처리하면 된다.
    # if cnt == 488:
    #     break
    cnt += 1
    
    # quality score ranges from 0-100, a higher score indicates a better quality
    # 최종적으로 예측 점수와 몇 번째 이미지인지 나오게 된다.
    print('Predicted quality score: %.2f, %d' % (score, cnt))

C:\Users\wodud\AppData\Local\Temp\ipykernel_17240\855171706.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img.cpu()).unsqueeze(0)


Predicted quality score: 76.78, 1
Predicted quality score: 68.98, 2
Predicted quality score: 33.85, 3


In [33]:
# 결과
df

,Model_name,filename,score
0,HyperIQA,D_01.jpg,76.782438
1,HyperIQA,D_02.jpg,68.978039
2,HyperIQA,D_03.jpg,33.845548
